First step is import data from Parse.ly's AWS bucket for the relevant  time period into Google BigQuery.  
The key and passphrase for these are with rachit.kinger@jpimedia.co.uk. More key and passwords can be obtained by reaching out directly to Parse.ly.  
This data is best imported via GCP's GUI. Go to `GCP > Storage > Transfer` and the steps from there should be obvious.  
Remember to specify file filters to download data only for the relevant months.  

Once this data has been download the follow these steps to import into a single 'large' database. DO NOT use this database for analysis. Only around 30% of this dataset is useful for us. This database will have a column called `action` which has mostly two values, `pageview` or `heartbeat`. The `pageview` is what is of interest to us and it occupies only 30% of the database.  



In [ ]:
# the following commands were run from bash using gsutil
# create empty table with the rawdata schema
%%bash
bq mk -t --schema /home/rachit/gdrive/GCP/bigquery_parsely/bqtable_from_cli/parsely_rawdata_schema.json \
--time_partitioning_type=DAY --time_partitioning_field ts_action \
--require_partition_filter=TRUE --clustering_fields='apikey,action' \
parsely.rawdata

In [ ]:
# the following commands were run from bash using gsutil
# use the following command to load the raw parsely data into the the empty table created in previous step
%%bash
bq load --source_format=NEWLINE_DELIMITED_JSON --max_bad_records=1000 --ignore_unknown_values parsely.rawdata gs://parsely/*

In [1]:
# use oauth to authenticate access into GCP and bigquery
# toggle this variable depending on whether you are on a machine that has an Internet browser or not
launch_browser = False 
# The `launch_browser` boolean variable indicates if a local server is used
# as the callback URL in the auth flow. A value of `True` is recommended,
# but a local server does not work if accessing the application remotely,
# such as over SSH or from a remote Jupyter notebook.

# if you are not Rachit Kinger please build your own oauth access client id and key and set the 
# path in the function below to your own client ke
from google_auth_oauthlib import flow

appflow = flow.InstalledAppFlow.from_client_secrets_file(
    '/home/rachit/gdrive/GCP/oauth_client_key/data-team-rachit-kinger-desktop-apps.json',
    scopes = ['https://www.googleapis.com/auth/bigquery'])

if launch_browser:
    appflow.run_local_server()
else:
    appflow.run_console()
    
credentials = appflow.credentials

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=306244342965-pv1n0vhib5nv2ks66pnj59csdf8cqor0.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=m36ZbrRtyeg4cEzKUUUf62TuDRcXTv&prompt=consent&access_type=offline


Enter the authorization code:  4/_ADXejaFJx7AY1KqZySJOVYHxzPFulljKCFpx57b_NXE-2DN7RaHxNQ


In [4]:
# access bigquery
from google.cloud import bigquery
project_id = "bigquery-test-165213"
client = bigquery.Client(project = project_id, credentials = credentials)